In [1]:
from data_processing import csv_data, write_to_json_file, get_temperature_by_day
from forecasting import arima_estimation, sarima_estimation
from datetime import datetime, timedelta
from statsmodels.tsa.arima.model import ARIMA
import time
import json

In [2]:
dataClass = csv_data("../data/urgence3.csv")

c:\users\sofiane\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3347: DtypeWarning: Columns (11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
with open("../data/hourly_temperature.json", 'r') as f:
    temp_dict = json.load(f)

In [4]:
start_d, start_m, start_y = 1, 1, 2019

In [5]:
path = "../data/predictions4/"

prev_week_load_file = "prev_week_load_temp.json"
prediction_file_name = "predictions_temp.json"

temp_path = "hourly_temperature.json"

In [6]:
n_prev_days = 7
prev_week_load = dataClass.return_prev_days(start_y, start_m, start_d, n_prev_days = n_prev_days)
index = dataClass.find_start_day(start_y, start_m, start_d)

In [7]:
date = datetime(start_y, start_m, start_d, 0, 0)
def get_temp_prev_days(date, n_prev_days = 7):
    date_copy = date - timedelta(days = n_prev_days)
    temps = []
    for i in range(n_prev_days):
        dic = get_temperature_by_day(date_copy)
        temps += dic[list(dic.keys())[0]]
        date_copy += timedelta(days = 1)
    return temps

def get_temp_next_days(date, n_next_days = 7):
    pred_temp = []
    date_copy = date
    for i in range(n_next_days):
        dic = get_temperature_by_day(date_copy)
        pred_temp += dic[list(dic.keys())[0]]
        date_copy += timedelta(days = 1)
    return pred_temp

In [8]:
pred_temps = get_temp_next_days(date)

In [9]:
temps = get_temp_prev_days(date, 7)

In [10]:
stop = 1
interval = 2

dic_key = f"{start_y:02}-{start_m:02}-{start_d:02}"
prev_week_load_dict = {dic_key:prev_week_load}
prev_index = index
prev_week_load_dict[dic_key].append(0)
n_samples = 100

# today_temps_dic = get_temperature_by_day(date)
# today_temps = today_temps_dic[list(today_temps_dic.keys())[0]]
temps.append(pred_temps.pop(0))
n_pred_steps = 24

while(True and index >= 0):
    date += timedelta(minutes = interval)
    time.sleep(1)
    
    while(datetime.strptime(dataClass.dates[index], dataClass.dateFormat) < date):
        prev_week_load_dict[dic_key][-1] += 1
        index -= 1
    if(prev_index != index):
        write_to_json_file(path, prev_week_load_file, prev_week_load_dict)
        model = sarima_estimation(prev_week_load_dict[dic_key][-n_samples:], exog = temps[-n_samples:])
        print(dataClass.dates[index], model.aic)
        try:
            predictions = model.forecast(steps = n_pred_steps, exog = pred_temps[0:n_pred_steps])
            predictions_dict = {f"{start_y:02}-{start_m:02}-{start_d:02}": list(map(round, predictions))}
            write_to_json_file(path, prediction_file_name, predictions_dict)
        except:
            raise ValueError("Cannot forecast !!")
    
    prev_index = index
    if(date.minute == 0):
        prev_week_load_dict[dic_key].append(0)
        temps.append(pred_temps.pop(0))
    
#     print(index, dataClass.dates[index], date)

01/01/2019 00:14 410.9111877407042
01/01/2019 00:17 410.06771128788876
01/01/2019 00:55 409.43646514862814
01/01/2019 00:56 409.0234855042141
01/01/2019 00:59 408.86230385348597


c:\users\sofiane\appdata\local\programs\python\python37\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


01/01/2019 01:02 409.11162304598963
01/01/2019 01:06 408.8899928857351
01/01/2019 01:09 408.57051285889287
01/01/2019 01:32 408.4599640368162


KeyboardInterrupt: 